# Test Qlora

based on [Medium Article](https://medium.com/p/13a7d3a256f6)

In [ ]:
# Set those token to clone repo and access huggingface
GIT_TOKEN = ''
HUGGINGFACE_TOKEN = ''

repo = 'SwissTextSDG'
!rm -r SwissTextSDG/
!git clone https://{GIT_TOKEN}@github.com/vera-bernhard/{repo}


In [ ]:
%cd /content/SwissTextSDG/
!git checkout qlora

In [ ]:
!pip install transformers
!pip install accelerate
!pip install peft
!pip install datasets
!pip install bitsandbytes
!pip install swifter
!pip install pytorch_transformers
!pip install huggingface_hub

from huggingface_hub import login
login(token=HUGGINGFACE_TOKEN)

In [24]:
import torch
from transformers import (AutoTokenizer,
                          MistralForSequenceClassification, 
                          BitsAndBytesConfig, 
                          Trainer, 
                          TrainingArguments)
from datasets import load_dataset
from peft import (LoraConfig, 
                  PeftConfig, 
                  PeftModel, 
                  get_peft_model,
                  prepare_model_for_kbit_training)
from sklearn.metrics import f1_score

In [25]:
model_checkpoint = 'mistralai/Mistral-7B-v0.1'

In [26]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= 'nf4',
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= True,
)

In [ ]:
model = MistralForSequenceClassification.from_pretrained(
        model_checkpoint,
        num_labels=18,
        quantization_config=bnb_config,
        device_map='auto',
)

In [ ]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=2,
    bias='none',
    task_type='SEQ_CLS',
    target_modules=['q_proj', 'v_proj']
)
model = get_peft_model(model, peft_config)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

In [1]:
%cd /content/SwissTextSDG

/home/vera/Documents/Uni/Master/FS24/ML4NLP2/Project/SwissTextSDG


In [ ]:
import sys
import os

from src.data.dataset import TrainSwissTextDataset
     

dataset = TrainSwissTextDataset.create_instance(dataset='swisstext_task1_train', model_name='mbert', use_val=False, train_frac=0.33)
train_df, test_df, val_df = dataset.get_data_loaders()

In [7]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1_result = f1_score(labels, preds, average='weighted')
    return {'f1_score': f1_result}

In [ ]:
training_args = TrainingArguments(
    output_dir='./checkpoints',
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    num_train_epochs=1,
    weight_decay=0.01
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_df,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()